# Generic line coding

The _GenericLineCoding_ layer performs the following:

-   [__Encoding__] With a signal mapping provided in the constructor, applies the mapping to
    an input symbol stream using keys from the mapping, and produces either:

    -   a flattened 1-d stream of subsymbols,
    -   a 2-d array with shape: (symbol stream length, subsymbols * samples/subsymbol)

    The type of output is controlled with a 'flattened' parameter. An unflattened
    version can always be produced with the `_encode_as_subsymbols` method. Since
    it's more meaningful for this layer to return 1-d streams, the condition that
    `decode(encode(x)) == x` is not valid by default. The `_encode_as_subsymbols` can
    be used for dummy input or testing.

-   [__Decoding__] Takes a 2-d array of samples, where each row should contain a number of
    samples meant to represent a single symbol, i.e. should contain all subsymbols. A
    mask is used with phase-offseted 'carrier' symbols.

Constructor arguments:
- "signal" (required) the signal definition, with integer symbols and a 'carrier' key,
- "samples_per_subsymbol" (optional, default: 2),
- "flattened" (optional, default: True).

Preconditions:
- Encode input is a NumPy array,
- Encode input is a 1-d stream,
- Encode input has only values defined in the signal mapping (signal sans carrier),
- Decode input is a 2-d array of valid width.

Postconditions:
- Encode output is a 1-d stream or a 2-d array of valid width,
- Decode output is a 1-d stream,
- Decode output has only values defined in the signal mapping.

In [1]:
import sys
sys.path.append('..')
%pylab inline
%precision 3
numpy.set_printoptions(linewidth=100)
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Populating the interactive namespace from numpy and matplotlib


In [2]:
import datapro
from datapro.layer import *
from datapro.util.scinum import *

## Basics

### Signals & masks

Decoding uses phase-offsetted masks generated from a _carrier_ supplied with the signal.

In [3]:
signal = {0: [1, 0], 1: [0, 1], "carrier": [1, 0]}
coder = lne.GenericLineCoding(signal=signal)

Once a signal is provided, the following properties are available in the coder:
1. _signal_
2. _mapping_
3. _carrier_

In [4]:
print("signal: ", coder.signal)
print("mapping:", coder.mapping)
print("carrier:", coder.carrier)

signal:  {0: [1, 0], 1: [0, 1], 'carrier': [1, 0]}
mapping: {0: [1, 0], 1: [0, 1]}
carrier: [1, 0]


Masks are created using phase-offseted copies of the carrier. The number of phases is:
$phases = 2 \times (symcount - 1)$. The carrier is offseted using the `np.roll` function. Uneven sizes are permitted, and produced using interpolation, set to _nearest_ by default.

In [5]:
coder.create_mask()

array([[1., 1., 0., 0.],
       [0., 1., 1., 0.]])

In [6]:
coder.create_mask(samples=6)

array([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0.]])

Below is an example of a carrier that is of different length than the symbols. Only symbol → subsymbols mappings need to have same length.

In [7]:
coder.signal.update(carrier=[0.7, 1, 0.1, 0])
coder.signal = coder.signal # properties are updated in the property setter

In [8]:
coder.create_mask()

array([[0.7, 1. , 0.1, 0. ],
       [0. , 0.7, 1. , 0.1]])

In [9]:
coder.create_mask(samples=5, kind="linear")

array([[0.7  , 0.8  , 0.9  , 1.   , 0.7  , 0.4  , 0.1  , 0.067, 0.033, 0.   ],
       [0.033, 0.   , 0.7  , 0.8  , 0.9  , 1.   , 0.7  , 0.4  , 0.1  , 0.067]])

## Demo: _Manchester-like coding_

There is an alias for Manchester coding, `datapro.layer.lne.ManchesterLineCoding`, which defines the signal internally, such that it does not need to be provided manually.

### Encoding

Create random array with integer symbols in the range $[0, 2)$.

In [10]:
coder = lne.ManchesterLineCoding()
msg = np.random.randint(0, 2, 10, dtype=int)
/print msg

[1 1 0 0 1 0 1 1 0 1]


Since the coder produces flattened output by default, we need to use `_encode_as_subsymbols` to get `decode`-compatible output. The argument `samples` controls how many samples are used per subsymbol.

In [11]:
encoded_stream = coder.encode(msg)
encoded = coder._encode_as_subsymbols(msg, samples=4)
/print encoded_stream
/print
/print encoded

[0 1 0 1 1 0 1 0 0 1 1 0 0 1 0 1 1 0 0 1]

[[0 0 0 0 1 1 1 1]
 [0 0 0 0 1 1 1 1]
 [1 1 1 1 0 0 0 0]
 [1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]
 [1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]
 [0 0 0 0 1 1 1 1]
 [1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]]


### Decoding

For decoding, we need to provide at least the ideal symbol stream (in this case, the `msg`). If the _phase_ is equal to _train_, each subsequent call to `decode` will perform a partial fit of the internal naïve Bayes model. The runtime config is controlled via the property `config`. At minimum, the decoding requires the ideal symstream to be provided, via a `symstream` key (*the name was chosen such that a Run's o_streams dict can be given like: config = {**run.o_streams}*).

In [12]:
coder.config = dict(symstream=msg, phase="train")
/print coder.config

{'symstream': array([1, 1, 0, 0, 1, 0, 1, 1, 0, 1]), 'phase': 'train'}


For decoding we will add some gaussian noise to the signal using `datapro.util.scinum.add_awgn_noise`, with parameters $\sigma$ (_std. dev._) and $\mu$ (_mean_).

In [13]:
@interact(σ=(0.0, 2.0, 0.05), μ=(0.0, 10.0))
def decode(σ=0.0, μ=0.0):
    decoded = coder.decode(add_awgn_noise(encoded, σ, μ))
    print("decoded: ", decoded)
    print("original:", msg)
    print("errors:  ", count_errors(decoded, msg))

interactive(children=(FloatSlider(value=0.0, description='σ', max=2.0, step=0.05), FloatSlider(value=0.0, desc…

In [14]:
@interact_manual(σ=(0.0, 2.0, 0.05), μ=(0.0, 10.0), r=(0, 500, 20))
def evaluate(σ=1.0, μ=0.0, r=40):
    x = 0
    for i in range(r):
        x += count_errors(coder.decode(add_awgn_noise(encoded, σ, μ)), msg)

    print("σ: {}, μ: {}".format(σ, μ))
    print("reps:  ", r)
    print("errors: {}/{} ({:.2f}%)".format(x, len(msg) * r, 100 * x/r/len(msg)))

interactive(children=(FloatSlider(value=1.0, description='σ', max=2.0, step=0.05), FloatSlider(value=0.0, desc…

### Evaluation

The previous coder was configured in a _train_ phase. We can take its decision device and use it to evaluate another coder.

In [15]:
evalcoder = type(coder)()

The decision device can be either provided as runtime config (via the `config` property), or via a specific `decision_device` parameter.

In [16]:
evalcoder.config = dict(symstream=msg,
                        phase="eval",
                        decision_device=coder.decision_device)
# evalcoder.decision_device = coder.decision_device

In [17]:
@interact(σ=(0.0, 2.0, 0.05), μ=(0.0, 10.0))
def decode(σ=0.5, μ=0.0):
    decoded = evalcoder.decode(add_awgn_noise(encoded, σ, μ))
    errors = count_errors(decoded, msg)
    print("decoded: ", decoded)
    print("original:", msg)
    print("errors: {}/{} ({:.1f}%)".format(errors, len(msg), 100*errors/len(msg)))

interactive(children=(FloatSlider(value=0.5, description='σ', max=2.0, step=0.05), FloatSlider(value=0.0, desc…

## Demo: _Alternative signals_

With the GenericLineCoder we are not limited to simple signals.

In [18]:
altsignal = {
    0: [1, 0, 0],
    1: [0, 1, 0],
    2: [0, 0, 1],
    "carrier": [1, 1, 0, 0, 0, 0]
}
altcoder = lne.GenericLineCoding(signal=altsignal)
print("Subsymbol count:", altcoder.subsymbol_count)
print("Phase count:", altcoder.phase_count)

Subsymbol count: 3
Phase count: 4


Since the coder has a subsymbol count of 3, there are 4 phase-offseted carriers in the mask.

In [19]:
altcoder.create_mask()

array([[1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 1., 1., 0.]])

Random message with symbols from range [0, 2].

In [20]:
altmsg = np.random.randint(0, altcoder.subsymbol_count, 15, dtype=int)
/print altmsg

[1 0 0 1 1 2 0 0 0 1 1 2 2 0 2]


Encoded message with 2 samples per subsymbol.

In [21]:
altencoded = altcoder._encode_as_subsymbols(altmsg, samples=2)
/print altencoded

[[0 0 1 1 0 0]
 [1 1 0 0 0 0]
 [1 1 0 0 0 0]
 [0 0 1 1 0 0]
 [0 0 1 1 0 0]
 [0 0 0 0 1 1]
 [1 1 0 0 0 0]
 [1 1 0 0 0 0]
 [1 1 0 0 0 0]
 [0 0 1 1 0 0]
 [0 0 1 1 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 1]
 [1 1 0 0 0 0]
 [0 0 0 0 1 1]]


The symbol space has the length of the message and width equal to number of phases.

In [22]:
/print altcoder.create_symbol_space(altencoded)

[[-0.667  0.333  1.333  0.333]
 [ 1.333  0.333 -0.667 -0.667]
 [ 1.333  0.333 -0.667 -0.667]
 [-0.667  0.333  1.333  0.333]
 [-0.667  0.333  1.333  0.333]
 [-0.667 -0.667 -0.667  0.333]
 [ 1.333  0.333 -0.667 -0.667]
 [ 1.333  0.333 -0.667 -0.667]
 [ 1.333  0.333 -0.667 -0.667]
 [-0.667  0.333  1.333  0.333]
 [-0.667  0.333  1.333  0.333]
 [-0.667 -0.667 -0.667  0.333]
 [-0.667 -0.667 -0.667  0.333]
 [ 1.333  0.333 -0.667 -0.667]
 [-0.667 -0.667 -0.667  0.333]]


In [23]:
@interact(σ=(0.0, 2.0, 0.05), μ=(0.0, 10.0))
def altdecode(σ=0.5, μ=0.0):
    altcoder.config = dict(symstream=altmsg, phase="run")
    altdecoded = altcoder.decode(add_awgn_noise(altencoded, σ, μ))
    errors = count_errors(altdecoded, altmsg)
    print("decoded: ", altdecoded)
    print("original:", altmsg)
    print("errors: {}/{} ({:.1f}%)".format(errors, len(altmsg), 100*errors/len(altmsg)))

interactive(children=(FloatSlider(value=0.5, description='σ', max=2.0, step=0.05), FloatSlider(value=0.0, desc…

## Demo: _Saturation and symbol replacement_

Since in some cases exact values are only known in subsequent layers, the GenericLineCoding layer allows replacing certain output subsymbols with another. The _saturated_ property is False by default. The replacement policy is controlled by an `__init__` keyword argument *saturate_mapping* or a property `saturate_mapping` at runtime.

In [24]:
satcoder = lne.ManchesterLineCoding()
satmsg = np.random.randint(0, 2, 10, dtype=int)

print("saturated:", satcoder.saturated)
print("saturate_mapping:", satcoder.saturate_mapping)

saturated: False
saturate_mapping: {1: -1}


Normally, no replacement is performed.

In [25]:
print(satcoder.encode(satmsg))

[1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 0 1 0 0 1]


Once we set _saturated_ to true, the replacement will be performed.

In [26]:
satcoder.saturated = True
print(satcoder.encode(satmsg))

[-1  0 -1  0  0 -1  0 -1  0 -1 -1  0 -1  0 -1  0 -1  0  0 -1]


The mapping can be changed at runtime too. For compatibility with serialised configs, string keys are transformed with the `int` function.

In [27]:
satcoder.saturate_mapping = {'0': -1, 1: 8}
print("saturate_mapping:", satcoder.saturate_mapping, "\n")
print(satcoder.encode(satmsg))

saturate_mapping: {0: -1, 1: 8} 

[ 8 -1  8 -1 -1  8 -1  8 -1  8  8 -1  8 -1  8 -1  8 -1 -1  8]


The saturation works also at the subsymbol level when the layer produces 2-d values.

In [28]:
satcoder.flattened = False
print(satcoder.encode(satmsg))

[[ 8  8 -1 -1]
 [ 8  8 -1 -1]
 [-1 -1  8  8]
 [-1 -1  8  8]
 [-1 -1  8  8]
 [ 8  8 -1 -1]
 [ 8  8 -1 -1]
 [ 8  8 -1 -1]
 [ 8  8 -1 -1]
 [-1 -1  8  8]]
